In [4]:
!pip install -q -U tensor2tensor
!pip install -q tensorflow matplotlib
!pip install tensorflow==1.14.0
!pip install tensor2tensor==1.11.0
!pip install tensorflow-probability==0.7.0
!pip install pydub
!apt-get -qq update
!apt-get -qq install -y ffmpeg
!apt-get -qq install -y sox

     |████████████████████████████████| 421.8MB 31kB/s 
     |████████████████████████████████| 3.8MB 35.4MB/s 
ERROR: tensorboard 2.1.0 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
  Using cached https://files.pythonhosted.org/packages/d3/59/d88fe8c58ffb66aca21d03c0e290cd68327cc133591130c674985e98a482/tensorflow-1.14.0-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/3c/d5/21860a5b11caf0678fbc8319341b0ae21a07156911132e0e71bffed0510d/tensorflow_estimator-1.14.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f4/37/e6a7af1c92c5b68fb427f853b06164b56ea92126bcfd87784334ec5e4d42/tensorboard-1.14.0-py2-none-any.whl
ERROR: tensorflow-addons 0.7.1 has requirement tensorflow>=2.1.0, but you'll have tensorflow 1.14.0 which is incompatible.
  Found existing installation: tensorflow-estimator 2.1.0
    Uninstalling tensorflow-estimator-2.1.0:
      Successfully uninstalled tensorflow-estimat

  Using cached https://files.pythonhosted.org/packages/ea/a9/c10b5283edbea85f4eb5a99d58a9edb43e764075dbc3fb50cae979ade845/tensor2tensor-1.11.0-py2.py3-none-any.whl
  Found existing installation: tensor2tensor 1.15.7
    Uninstalling tensor2tensor-1.15.7:
      Successfully uninstalled tensor2tensor-1.15.7


In [1]:
import sys
if 'google.colab' in sys.modules: # Colab-only TensorFlow version selector
  %tensorflow_version 1.x
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import collections
import base64
import cStringIO
import pydub
import shutil
from scipy.io import wavfile

import IPython
import google.colab

from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.layers import common_layers
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import t2t_model
from tensor2tensor.utils import registry
from tensor2tensor.utils import metrics

# Enable TF Eager execution
tfe = tf.contrib.eager
tf.enable_eager_execution()

# Other setup
Modes = tf.estimator.ModeKeys

# Setup some directories
data_dir = os.path.expanduser("~/t2t/data")
tmp_dir = os.path.expanduser("~/t2t/tmp")
train_dir = os.path.expanduser("~/t2t/train")
checkpoint_dir = os.path.expanduser("~/t2t/checkpoints")
tf.gfile.MakeDirs(data_dir)
tf.gfile.MakeDirs(tmp_dir)
tf.gfile.MakeDirs(train_dir)
tf.gfile.MakeDirs(checkpoint_dir)

gs_ckpt_dir = "gs://tensor2tensor-checkpoints/"

TensorFlow 1.x selected.


W0724 15:37:47.127235 140417261496192 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/tensor2tensor/utils/expert_utils.py:68: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0724 15:37:48.038988 140417261496192 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0724 15:37:49.676784 140417261496192 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/tensor2tensor/utils/adafactor.py:27: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0724 15:37:49.677388 140417261496192 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/d

In [2]:
problem_name = "librispeech_clean"
asr_problem = problems.problem(problem_name)
encoders = asr_problem.feature_encoders(None)

model_name = "transformer"
hparams_set = "transformer_librispeech_tpu"

hparams = trainer_lib.create_hparams(hparams_set,data_dir=data_dir, problem_name=problem_name)



In [11]:
import json
print json.loads(hparams.to_json())
hparams.learning_rate = 0.1
hparams.batch_size = 50


{u'moe_loss_coef': 0.001, u'clip_grad_norm': 0.0, u'layer_prepostprocess_dropout': 0.2, u'multiply_embedding_mode': u'sqrt_depth', u'optimizer_adafactor_multiply_by_parameter_scale': True, u'multiproblem_max_target_length': -1, u'attention_value_channels': 0, u'moe_overhead_train': 1.0, u'factored_logits': False, u'audio_num_mel_bins': 80, u'compress_steps': 0, u'num_heads': 2, u'multiproblem_schedule_threshold': 0.5, u'num_encoder_layers': 6, u'conv_first_kernel': 9, u'self_attention_type': u'dot_product', u'max_target_seq_length': 350, u'attention_dropout_broadcast_dims': u'0,1', u'proximity_bias': False, u'shared_embedding_and_softmax_weights': True, u'learning_rate_schedule': u'rsqrt_decay', u'optimizer_adafactor_factored': True, u'moe_num_experts': 16, u'ffn_layer': u'conv_relu_conv', u'moe_k': 2, u'multiproblem_mixing_schedule': u'constant', u'optimizer_adam_epsilon': 1e-09, u'symbol_dropout': 0.0, u'scheduled_sampling_prob': 0.0, u'use_fixed_batch_size': False, u'initializer': u

In [12]:
asr_model = registry.model(model_name)(hparams, Modes.PREDICT)

def encode(x):
  waveforms = encoders["waveforms"].encode(x)
  encoded_dict = asr_problem.preprocess_example({"waveforms":waveforms, "targets":[]}, Modes.PREDICT, hparams)
  
  return {"inputs" : tf.expand_dims(encoded_dict["inputs"], 0), "targets" : tf.expand_dims(encoded_dict["targets"], 0)}

def decode(integers):
  integers = list(np.squeeze(integers))
  if 1 in integers:
    integers = integers[:integers.index(1)]
  return encoders["targets"].decode(np.squeeze(integers))

W0724 16:00:53.992930 140417261496192 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/tensor2tensor/utils/t2t_model.py:1905: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.



:::MLPv0.5.0 transformer 1595606454.523003101 (/usr/local/lib/python2.7/dist-packages/tensor2tensor/models/transformer.py:59) model_hp_embedding_shared_weights: {"vocab_size": 256, "hidden_size": 384}


I0724 16:00:54.523099 140417261496192 mlperf_log.py:156] :::MLPv0.5.0 transformer 1595606454.523003101 (/usr/local/lib/python2.7/dist-packages/tensor2tensor/models/transformer.py:59) model_hp_embedding_shared_weights: {"vocab_size": 256, "hidden_size": 384}
W0724 16:00:54.528589 140417261496192 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/tensor2tensor/layers/common_layers.py:75: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0724 16:00:54.553909 140417261496192 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/tensor2tensor/utils/t2t_model.py:167: The name tf.summary.text is deprecated. Please use tf.compat.v1.summary.text instead.



In [13]:
# Copy the pretrained checkpoint locally
ckpt_name = "transformer_asr_180214"
gs_ckpt = os.path.join(gs_ckpt_dir, ckpt_name)
print(gs_ckpt)
!gsutil cp -R {gs_ckpt} {checkpoint_dir} 
ckpt_path = tf.train.latest_checkpoint(os.path.join(checkpoint_dir, ckpt_name))
ckpt_path

gs://tensor2tensor-checkpoints/transformer_asr_180214
Copying gs://tensor2tensor-checkpoints/transformer_asr_180214/checkpoint...
Copying gs://tensor2tensor-checkpoints/transformer_asr_180214/model.ckpt-230000.data-00000-of-00001...
Copying gs://tensor2tensor-checkpoints/transformer_asr_180214/model.ckpt-230000.index...
Copying gs://tensor2tensor-checkpoints/transformer_asr_180214/model.ckpt-230000.meta...
/ [4 files][290.3 MiB/290.3 MiB]                                                
Operation completed over 4 objects/290.3 MiB.                                    


u'/root/t2t/checkpoints/transformer_asr_180214/model.ckpt-230000'

In [14]:
# Restore and transcribe!
def transcribe(inputs):
  encoded_inputs = encode(inputs)
  with tfe.restore_variables_on_create(ckpt_path): 
    model_output = asr_model.infer(encoded_inputs, beam_size=2, alpha=0.6, decode_length=1)["outputs"]
  return decode(model_output)

def play_and_transcribe(inputs):
  waveforms = encoders["waveforms"].encode(inputs)
  IPython.display.display(IPython.display.Audio(data=waveforms, rate=16000))
  return transcribe(inputs) 

Record your own voice

In [15]:
# Records webm file and converts
def RecordNewAudioSample(filename=None, webm_filename=None):
  """Args:
          filename - string, path for storing wav file
          webm_filename - string, path for storing webm file
     Returns:
          string - path where wav file was saved. (=filename if specified)
    
  """
  # Create default filenames in tmp_dir if not specified.
  if not filename:
    filename = os.path.join(tmp_dir, "recording.wav")
  if not webm_filename:
    webm_filename = os.path.join(tmp_dir, "recording.webm")
    
  # Record webm file form colab.
  
  audio = google.colab._message.blocking_request('user_media', {"audio":True, "video":False, "duration":-1}, timeout_sec=600)
  #audio = frontend.RecordMedia(True, False)
  
  # Convert the recording into in_memory file.
  music_mem_file = cStringIO.StringIO(
      base64.decodestring(audio[audio.index(',')+1:]))
  
  # Store webm recording in webm_filename. Storing is necessary for conversion.
  with open(webm_filename, 'w') as fd:
    music_mem_file.seek(0)
    shutil.copyfileobj(music_mem_file, fd)
    
  # Open stored file and save it as wav with sample_rate=16000.
  pydub.AudioSegment.from_file(webm_filename, codec="opus"
                              ).set_frame_rate(16000).export(out_f=filename,
                                                             format="wav")
  return filename

Talk here talk here ! Click the cell next to this text :)

In [28]:
# Record the sample
my_sample_filename = RecordNewAudioSample()
print my_sample_filename

/root/t2t/tmp/recording.wav


Now Click it and see what you said :)

In [29]:
print play_and_transcribe(my_sample_filename)

:::MLPv0.5.0 transformer 1595619358.924473047 (/usr/local/lib/python2.7/dist-packages/tensor2tensor/utils/expert_utils.py:231) model_hp_layer_postprocess_dropout: 0.0


I0724 19:35:58.924563 140417261496192 mlperf_log.py:156] :::MLPv0.5.0 transformer 1595619358.924473047 (/usr/local/lib/python2.7/dist-packages/tensor2tensor/utils/expert_utils.py:231) model_hp_layer_postprocess_dropout: 0.0


:::MLPv0.5.0 transformer 1595619359.301307917 (/usr/local/lib/python2.7/dist-packages/tensor2tensor/models/transformer.py:101) model_hp_hidden_layers: 6


I0724 19:35:59.301345 140417261496192 mlperf_log.py:156] :::MLPv0.5.0 transformer 1595619359.301307917 (/usr/local/lib/python2.7/dist-packages/tensor2tensor/models/transformer.py:101) model_hp_hidden_layers: 6


:::MLPv0.5.0 transformer 1595619359.385380983 (/usr/local/lib/python2.7/dist-packages/tensor2tensor/models/transformer.py:101) model_hp_attention_dropout: 0.0


...

:::MLPv0.5.0 transformer 1595619866.237524033 (/usr/local/lib/python2.7/dist-packages/tensor2tensor/models/transformer.py:153) model_hp_attention_dense: {"num_heads": 2, "use_bias": "false", "hidden_size": 384}


I0724 19:44:26.237576 140417261496192 mlperf_log.py:156] :::MLPv0.5.0 transformer 1595619866.237524033 (/usr/local/lib/python2.7/dist-packages/tensor2tensor/models/transformer.py:153) model_hp_attention_dense: {"num_heads": 2, "use_bias": "false", "hidden_size": 384}


:::MLPv0.5.0 transformer 1595619866.522489071 (/usr/local/lib/python2.7/dist-packages/tensor2tensor/models/transformer.py:153) model_hp_norm: {"hidden_size": 384}


I0724 19:44:26.522522 140417261496192 mlperf_log.py:156] :::MLPv0.5.0 transformer 1595619866.522489071 (/usr/local/lib/python2.7/dist-packages/tensor2tensor/models/transformer.py:153) model_hp_norm: {"hidden_size": 384}


FRIENDLY PROVIDENCE AS DISCUSSED IN THE PREVIOUS TWO SICTIONS WE SEE THAT ORDINARY GENERATING FUNCTIONS ARE EQUICABLE AIM DISTRIBUTION PROBLEMS OR ARRANGEMENT PROBLEMS IT


Upload pre-recorded samples

In [ ]:
uploaded = google.colab.files.upload()
prerecorded_messages = []

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  mem_file = cStringIO.StringIO(uploaded[fn])
    
  save_filename = os.path.join(tmp_dir, fn)
  with open(save_filename, 'w') as fd:
    mem_file.seek(0)
    shutil.copyfileobj(mem_file, fd)
  prerecorded_messages.append(save_filename)
         
              
for inputs in prerecorded_messages:
  outputs = play_and_transcribe(inputs)

  print("Inputs: %s" % inputs)
  print("Outputs: %s" % outputs)